In [1]:
%pip install selenium
%pip install pandas
%pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
from time import sleep
from urllib.parse import urljoin
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import json

/Users/nicolepi/Documents/Wing/find-the-expert/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def scrape_directory(url):
    driver = webdriver.Chrome()
    names = []
    bio_urls = []
    titles = []

    try:
        driver.get(url)

        while True:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "listing-item"))
            )

            faculty_listings = driver.find_elements(By.CLASS_NAME, "listing-item")
            for faculty in faculty_listings:
                try:
                    link_element = faculty.find_element(By.TAG_NAME, "a")

                    name = link_element.text

                    title_div = faculty.find_element(By.CLASS_NAME, "excerpt")
                    title = title_div.text

                    link = link_element.get_attribute('href')
                    bio_url = urljoin(url, link)

                    names.append(name)
                    titles.append(title)
                    bio_urls.append(bio_url)
                    print(f"Name: {name}, Title: {title}, URL: {bio_url}")
                except NoSuchElementException:
                    continue

            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, "Next"))
                )

                retries = 5
                for _ in range(retries):
                    try:
                        next_button.click()
                        break
                    except ElementClickInterceptedException:
                        sleep(1)
                else:
                    print("Failed to click the 'Next' button after several retries.")
                    break

            except (TimeoutException, NoSuchElementException):
                # print("No 'Next' button found, or it's not clickable.")
                break

    except TimeoutException:
        print("Timeout while waiting for page to load")
    finally:
        driver.quit()

    df = pd.DataFrame({'Name': names, 'Title': titles, 'url': bio_urls})

    return df

def extract_biographies(df):
    driver = webdriver.Chrome()

    # Create a new column 'Research Summary'
    df['Research Summary'] = None  # Initialize with None

    for index, row in df.iterrows():
        bio_url = row['url']
        try:
            driver.get(bio_url)
            full_bio_text = ""

            try:
                # Find all elements that could contain the required sections
                potential_sections = driver.find_elements(By.XPATH, "//*[self::h6 or self::p]")

                research_interests_started = False
                selected_publications_reached = False
                bio_text = ""

                # Iterate through all potential sections to extract the relevant text
                for section in potential_sections:
                    # Check if the current section is the "Research Interests" heading
                    if section.tag_name == "h6" and section.text == "Research Interests":
                        research_interests_started = True
                        continue

                    # Check if we've reached the "Selected Publications" heading
                    if section.tag_name == "h6" and section.text == "Selected Publications":
                        selected_publications_reached = True
                        break

                    # If we are within the "Research Interests" section, append the text
                    if research_interests_started and not selected_publications_reached:
                        bio_text += section.text + "\n"

                # Trim the last newline character
                bio_text = bio_text.rstrip()

                # Add the extracted text to the full_bio_text variable
                full_bio_text += bio_text

            except Exception as e:
                # Handle any errors that might occur during the extraction
                print(f"An error occurred while processing the bio section: {e}")

            # Assign the 'Research Summary' to the corresponding row in the DataFrame
            if full_bio_text:
                df.at[index, 'Research Summary'] = full_bio_text
            else:
                df.at[index, 'Research Summary'] = None

        except TimeoutException:
            print(f"Timeout while loading {bio_url}")
        except Exception as e:
            print(f"Failed to extract biography from {bio_url}: {e}")

    driver.quit()
    return df

url = "http://ealac.columbia.edu/people/faculty/"
df = scrape_directory(url)
df = extract_biographies(df)

Name: Nicholas Bartlett, Title: Assistant Professor of Contemporary Chinese Culture and Society, Barnard College, URL: http://ealac.columbia.edu/nicholas-bartlett/
Name: JM Chris Chang, Title: Lecturer, Mellon Fellow in the Society of Fellows in the Humanities, URL: http://ealac.columbia.edu/jm-chris-chang/
Name: Yasmin Cho, Title: Postdoctoral Research Scholar, URL: http://ealac.columbia.edu/yasmin-cho/
Name: Eunice Euna Chung, Title: Lecturer in Korean, URL: http://ealac.columbia.edu/eunice-chung/
Name: Michael Como, Title: Tōshū Fukami Associate Professor of Shinto Studies, URL: http://ealac.columbia.edu/michael-como/
Name: Shigeru Eguchi, Title: Senior Lecturer in Japanese, URL: http://ealac.columbia.edu/shigeru-eguchi/
Name: Bernard Faure, Title: Kao Professor of Japanese Religion, URL: http://ealac.columbia.edu/bernard-faure/
Name: Robert Harrist, Title: Jane and Leopold Swergold Professor of Chinese Art History, URL: http://ealac.columbia.edu/robert-harrist/
Name: Agnes Hsu-Tang

In [5]:
# Initialize the sentence embedder model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

# Assuming 'Research Summary' column contains the text data
df['Research Embedding'] = df['Research Summary'].apply(lambda x: model.encode([x])[0] if x else None)
gdf = df

In [6]:
gdf = df
# Assuming 'gdf' is your original DataFrame

# Convert the 'Research Embedding' column to a list of lists
gdf['Research Embedding'] = gdf['Research Embedding'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else None)

# Convert the DataFrame to a list of dictionaries
data_list = gdf.to_dict(orient='records')

# Save the list of dictionaries to a JSON file
with open('faculty_data_ealc.json', 'w') as json_file:
    json.dump(data_list, json_file)

In [7]:
# Load the JSON file back into a DataFrame
with open('faculty_data_ealc.json', 'r') as json_file:
    loaded_data_list = json.load(json_file)

df = pd.DataFrame(loaded_data_list)

# Now, gdf_loaded should have the 'Research Embedding' column in its original form
df.head()

,Name,Title,url,Research Summary,Research Embedding
0,Nicholas Bartlett,Assistant Professor of Contemporary Chinese Cu...,http://ealac.columbia.edu/nicholas-bartlett/,"Addiction and recovery, labor, civil society, ...","[-0.007219260558485985, 0.036075059324502945, ..."
1,JM Chris Chang,"Lecturer, Mellon Fellow in the Society of Fell...",http://ealac.columbia.edu/jm-chris-chang/,"Socialism, bureaucracy, knowledge production, ...","[-0.07688195258378983, 0.12053538858890533, -0..."
2,Yasmin Cho,Postdoctoral Research Scholar,http://ealac.columbia.edu/yasmin-cho/,"Materiality, mobilities, gender, space, buildi...","[0.09585871547460556, 0.097362220287323, -0.02..."
3,Eunice Euna Chung,Lecturer in Korean,http://ealac.columbia.edu/eunice-chung/,Language for Specific Purposes\nSecond Languag...,"[-0.0030238013714551926, 0.049819231033325195,..."
4,Michael Como,Tōshū Fukami Associate Professor of Shinto Stu...,http://ealac.columbia.edu/michael-como/,Transmission and diffusion of rituals and deit...,"[0.012397040612995625, 0.13433648645877838, -0..."


In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
proposal = ["""
medical image machine learning
            """]

proposal_embedding = model.encode(proposal)[0]

In [ ]:
# Calculate dot product for each row
df['Dot Product'] = df['Research Embedding'].apply(lambda x: np.dot(proposal_embedding, x) if x is not None else None)

# Sort DataFrame based on dot product scores in descending order
df_sorted = df.sort_values(by='Dot Product', ascending=False)

top_5 = df_sorted.head(5)

print(top_5.iloc[0]['Research Summary'])

top_5.head()

In [ ]:
residual = proposal_embedding - top_5.iloc[0]['Research Embedding']
# print(residual)
proposal_embedding = residual

In [ ]:
king = model.encode('king')
norm = np.linalg.norm(king)
king = king / norm

man = model.encode('man')
norm = np.linalg.norm(man)
man = man / norm

woman = model.encode('woman')
norm = np.linalg.norm(woman)
woman = woman / norm

queen = model.encode('queen')
norm = np.linalg.norm(queen)
queen = queen / norm

queen_test = king - man + woman
norm = np.linalg.norm(queen_test)
queen_test = queen_test / norm

print('\'king - man + woman\' match to \'king\': ', np.dot(king, queen_test))
print('\'king - man + woman\' match to \'queen\': ', np.dot(queen, queen_test))

In [ ]:
# Calculate dot product for each row
df['Dot Product'] = df['Research Embedding'].apply(lambda x: np.dot(proposal_embedding, x) if x is not None else None)

# Sort DataFrame based on dot product scores in descending order
df_sorted = df.sort_values(by='Dot Product', ascending=False)

top_5 = df_sorted.head(5)

print(top_5.iloc[0]['Research Summary'])

In [ ]:
df = df.drop(columns=['Dot Product'])
df.to_csv('faculty_data.csv', index=False)

In [ ]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException

def create_database():
    conn = sqlite3.connect('biographies.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS biographies (
            id INTEGER PRIMARY KEY,
            url TEXT UNIQUE,
            biography TEXT
        )
    ''')
    conn.commit()
    conn.close()

def insert_biography(url, biography):
    conn = sqlite3.connect('biographies.db')
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO biographies (url, biography) VALUES (?, ?)", (url, biography))
        conn.commit()
    except sqlite3.IntegrityError:
        print(f"URL already exists in database: {url}")
    finally:
        conn.close()

def extract_biographies(bio_urls):
    driver = webdriver.Chrome()

    for bio_url in bio_urls:
        try:
            driver.get(bio_url)
            full_bio_text = ""

            try:
                body_section = driver.find_element(By.CLASS_NAME, 'body.col-lg-7')
                full_bio_text += body_section.text + " "
            except NoSuchElementException:
                pass

            try:
                target_section = driver.find_element(By.CLASS_NAME, 'col-lg-7.entity.entity-paragraphs-item.paragraphs-item-content')
                full_bio_text += target_section.text
            except NoSuchElementException:
                pass

            if full_bio_text:
                print(f"Storing biography from {bio_url}")
                insert_biography(bio_url, full_bio_text)

        except TimeoutException:
            print(f"Timeout while loading {bio_url}")
        except Exception as e:
            print(f"Failed to extract biography from {bio_url}: {e}")

    driver.quit()

# Create the database and table
create_database()

# Assuming you have a list of bio URLs
bio_urls = ["your_list_of_bio_urls_here"]
extract_biographies(bio_urls)